In [1]:
import requests
import pandas as pd
import json

In [10]:
def get_req():

    url = 'https://dlmm-api.meteora.ag/pair/all_by_groups'

    params = {"sort_key" : "feetvlratio", "order_by": 'desc'}

    response = requests.get(url,params=params)
    
    return response

In [14]:
def get_df(response):

    df = pd.DataFrame()
    for group in response.json()['groups']:
        if df.empty:
            df = pd.DataFrame(group)
            df = pd.json_normalize(df['pairs'])
        else:
            df1 = pd.DataFrame(group)
            df1 = pd.json_normalize(df1['pairs'])
            df = pd.concat([df, df1])
            
    return df

In [17]:
def get_pools(df):

    df = df[['name','bin_step','base_fee_percentage','liquidity','trade_volume_24h','apr', 'apy']]

    df['liquidity'] = df['liquidity'].astype(float)
    df['apr'] = df['apr'].astype(float)

    min_liquidity = 1000
    min_apr = 30

    cond1 = df['liquidity'] > min_liquidity
    cond2 = df['apr'] > min_apr
    df = df[cond1 & cond2]

    df.sort_values(by='apr', inplace = True, ascending = False)
    df.reset_index(drop = True, inplace=True)

    print(df.head(15))

In [18]:
if __name__ == "__main__":
    response = get_req()
    
    df = get_df(response)
    
    get_pools(df)

              name  bin_step base_fee_percentage      liquidity  \
0         NEET-SOL       100                   2    1779.966212   
1         NEET-SOL       250                   1    2156.293432   
2         FWOG-SOL       100                   2    4632.444986   
3   PUPPERFISH-SOL       100                   1    5336.409962   
4       CHERRY-SOL       100                   5   11814.923407   
5       Neiro-USDC       200                   1   11381.335945   
6        Neiro-SOL       125                   1    6356.903208   
7         FWOG-SOL       100                   1  185640.236101   
8     MANIFEST-SOL        80                 0.2    1954.408851   
9   PUPPERFISH-SOL        80                   5    9598.037486   
10        SOL-FWOG       400                   1    7384.404181   
11        FWOG-SOL       100                   5   38369.873820   
12        SOL-FWOG       250                   2  137790.261554   

    trade_volume_24h         apr           apy  
0       1.80

C:\Users\might\AppData\Local\Temp\ipykernel_27296\1974525312.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['liquidity'] = df['liquidity'].astype(float)
C:\Users\might\AppData\Local\Temp\ipykernel_27296\1974525312.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['apr'] = df['apr'].astype(float)
